### Задание 1:

Определить сочетание товаров в заказах и рассчитать для каждого сочетания количество раз, которое оно встречается. Выделить ТОП-20 наиболее популярных сочетаний

Вариант 1 (ищем самые популярные сочетания товаров (пары, тройки, четверки и пр)): 
Логика выполнения задания:

1. Предобработать данные
2. Получить все возможные сочетания товаров в чеке (пары, тройки, четверки и пр)
3. Посчитать количество таких сочетаний
4. Определить ТОП-20

Вариант 2: Упрощенная логика (ищем только самые популярные пары товаров)

Примечание: 

Для первоначального выполнения задания, количество купленного товара не учитывается

#### Вариант 1 (ищем самые популярные сочетания товаров (пары, тройки, четверки и пр)

In [1]:
# Импортируем библиотеки

import pandas as pd
import numpy as np
import itertools
from itertools import combinations

In [2]:
#Загрузим данные 

excel_data_df = pd.read_excel('тестовое задание.xlsx', sheet_name='корзина')

In [3]:
# Преобразуем таблицу

# Удалим лишние столбцы
excel_data_df.drop(excel_data_df.columns[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], axis = 1, inplace = True)

#Переименуем столбцы
excel_data_df.rename(columns = {'Номер заказа':'order_id', 'Номенклатура.Наименование':'name',

                              'Количество':'quantity'}, inplace = True)

In [4]:
# Проверим загруженные данные

excel_data_df

,order_id,name
0,16425,Бразильский орех
1,16425,Грецкий орех очищенный
2,16425,Изюм Голд
3,16425,Изюм Малаяр
4,16425,Изюм Султана
...,...,...
45609,20649,"Смесь ""Рататуй"" свежемороженая, Vитамин, 400г"
45610,20649,"Сотэ с прованскими травами свежемороженое, Vит..."
45611,20649,"Тайская смесь свежемороженая, Vитамин, 400г"
45612,20649,Тыква Гитара (баттернат)


Так как в задании необходимо определить наиболее полурные сочетания товаров, то при предобработке данных я буду учитывать следующиее:

1. Товар, который был куплен только один раз не может входить в самое популярное сочетание
2. Чеки с одним товаром, также не могут входить в самое популярное сочетание, так как в чеке только единичная позиция

In [5]:
# Выберем все товары, которые встречались только один раз 

count_quantity = excel_data_df.groupby('name').agg({'order_id' : 'count'}).sort_values(by = 'order_id').reset_index()

options_1 = count_quantity.query("order_id == 1")

options_1 = options_1['name'].tolist()

# Иcключим полученные значения из df

df_without_single_position = excel_data_df.loc[~excel_data_df['name'].isin(options_1)]

In [6]:
# Выберем все чеки, в которых только один товар 

count_receipts = excel_data_df.groupby('order_id').agg({'name' : 'count'}).sort_values(by = 'name').reset_index()

options_2 = count_receipts.query("name == 1")

options_2 = options_2['order_id'].tolist()

# Иcключим полученные значения из df

prepared_dataframe = df_without_single_position.loc[~df_without_single_position['order_id'].isin(options_2)]

#prepared_dataframe

In [7]:
# Создадим список из всех позиций в чеке 

prepared_dataframe_list = prepared_dataframe.groupby('order_id')['name'].apply(list)

# Преобразуем в датафрейм
df = pd.DataFrame(prepared_dataframe_list)

df = df.reset_index()

df

,order_id,name
0,15690,"[Арахис очищенный сырой, Грецкий орех очищенны..."
1,16345,"[Абрикосы молдавские, Базилик красный, Баклажа..."
2,16407,"[Авокадо ХАСС, Виноград зеленый без косточек, ..."
3,16425,"[Бразильский орех, Грецкий орех очищенный, Изю..."
4,16472,"[Апельсины столовые, Бананы, Вишня садовая, Гр..."
...,...,...
3193,ФББТ-000009,"[Ананасовые кольца, Арахис ""Ёжик"" в сахаре с к..."
3194,ФББТ-000010,"[Ананас кубики микс цукаты, Арахис в кокосовой..."
3195,ФББТ-015416,"[Абрикосы узбекские ""Шалах"", Ананасовые кольца..."
3196,ФББТ-015417,"[Ананас кубики микс цукаты, Ананасовые кольца,..."


Получили новый датафрейм, в котором у нас содержатся id чеков и списки всех товаров в нем. Создадим функцию, которая ,будет разворачивать список позиций чека и составлять возможные комбинации товаров (пары, тройки, четверки и тд), и преобразует полученные данные в датафрейм. 

Примечание:

Не смотря на то, что мы  оставили только те чеки, где количество позиций больше 1, и таких чеков 3198 шт, у нас есть чеки где количество позиций более 10, то есть с каждым чеком количество возможных комбинаций кратно увеличивается и для работы с таким объемом данных требуются приличные вычислительные мощности, которыми не обладает мой компьютер. Рассмотрим на примере нескольких чеком алгоритм нахождения ТОП-20 сочетаний продуктов в чеке. 

In [32]:
# Выберем несколько чеков из полученного df

df_small = df.query("order_id in ('19054', '19147', '19156', '19780')")

df_small

,order_id,name
1846,19054,"[Арбуз, Арбуз овальный, Дыня Колхозница, поздн..."
1928,19147,"[Арбуз, Дыня Торпеда узбекская]"
1936,19156,"[Сироп Ваниль Monin, 1л, Сироп Гренадин Monin,..."
2497,19780,"[Баклажаны грунтовые, Нектарины, Помидоры грун..."


In [33]:
# Задаем функцию описанную выше

d = pd.DataFrame() # Пустой датафрейм

for j in df_small['name']:
    output = pd.DataFrame(sum([list(map(list, combinations(j, i))) for i in range(len(j) + 1)], []))
    d = pd.concat([d,output])

In [35]:
#Удалим строки где все значения == Nan, которые возникли при генерации сочетаний

d = d.dropna(how='all')

# Удалим все строки, где содержится только одно значение, согласно логике выше. То есть оставим только те, 
# где вторая колонка не Nan

d = d[d[1].notna()]

d

,0,1,2,3,4,5
5,Арбуз,Арбуз овальный,None,None,NaN,NaN
6,Арбуз,"Дыня Колхозница, поздний сорт",None,None,NaN,NaN
7,Арбуз,Дыня Торпеда узбекская,None,None,NaN,NaN
8,Арбуз овальный,"Дыня Колхозница, поздний сорт",None,None,NaN,NaN
9,Арбуз овальный,Дыня Торпеда узбекская,None,None,NaN,NaN
...,...,...,...,...,...,...
59,Баклажаны грунтовые,Нектарины,Помидоры грунтовые Краснодар,Помидоры розовые крупные,Помидоры ростовские (Махитос),None
60,Баклажаны грунтовые,Нектарины,Помидоры Парадайз,Помидоры розовые крупные,Помидоры ростовские (Махитос),None
61,Баклажаны грунтовые,Помидоры грунтовые Краснодар,Помидоры Парадайз,Помидоры розовые крупные,Помидоры ростовские (Махитос),None
62,Нектарины,Помидоры грунтовые Краснодар,Помидоры Парадайз,Помидоры розовые крупные,Помидоры ростовские (Махитос),None


In [36]:
# Добавим еще один столбец, в котором будет находится перечень товаров из сочетания в формате str

d['combination'] = d[d.columns[0:]].apply(lambda x: ', '.join(x.dropna().astype(str).astype(str)),axis=1)

In [37]:
final_df = d.groupby('combination').agg({'combination' : 'count'})

final_df.rename(columns = {'combination':'combination', 'combination':'quantity',}, inplace = True)

final_df.sort_values(by = 'quantity', ascending = 0).head(20)

,quantity
combination,
"Арбуз, Дыня Торпеда узбекская",2
"Нектарины, Помидоры грунтовые Краснодар, Помидоры розовые крупные, Помидоры ростовские (Махитос)",1
"Нектарины, Помидоры грунтовые Краснодар, Помидоры Парадайз, Помидоры ростовские (Махитос)",1
"Нектарины, Помидоры грунтовые Краснодар, Помидоры Парадайз, Помидоры розовые крупные, Помидоры ростовские (Махитос)",1
"Нектарины, Помидоры грунтовые Краснодар, Помидоры Парадайз, Помидоры розовые крупные",1
"Нектарины, Помидоры грунтовые Краснодар, Помидоры Парадайз",1
"Нектарины, Помидоры грунтовые Краснодар",1
"Нектарины, Помидоры Парадайз, Помидоры ростовские (Махитос)",1
"Нектарины, Помидоры Парадайз, Помидоры розовые крупные, Помидоры ростовские (Махитос)",1


В нашем примере 'Арбуз, Дыня Торпеда узбекская' является самым популярным сочетанием

#### Заключение:

Основываясь на здравой логике, можно предположить, что шанс того, что сочетание из 6 товаров будет куда меньше, чем сочетание из двух, по этому при работе со всем массивом данных можно убрать из датафрейма все комбинации длиннее 6 элементов. 

#### Вариант 2 (ищем только самые популярные пары товаров)

Для нахождение самых популярных пар, я бы воспользовалась sql, запрос выглядем бы следующим образом 

In [ ]:
-- Тестовая временная таблица

WITH orders (order_id, "name") AS 
(
  VALUES (1, 'Бразильский орех')
  UNION ALL 
  VALUES (1, 'Грецкий орех очищенный')
  UNION ALL 
  VALUES (1, 'Изюм Голд')
  UNION ALL 
  VALUES (1, 'Имбирь сушеный в сахаре')
  UNION ALL 
  VALUES (2, 'Лук зеленый')
  UNION ALL 
  VALUES (2, 'Огурцы Миринда')
  UNION ALL 
  VALUES (2, 'Слива Президент')
), 
pair_preparation AS 
(
    SELECT  o1."name" name_1, 
            o2."name" name_2
    FROM    orders o1 
            CROSS JOIN orders o2
    WHERE   o1.order_id = o2.order_id
            AND o1."name" < o2."name" -- Используем < для того, чтобы исключить перестановки местами
    ORDER BY 
            1, 2
) 

SELECT  concat(name_1, ' ,', name_2) pair,
        count(concat(name_1, ' ,', name_2)) quantity
FROM    pair_preparation
GROUP BY 
        pair
ORDER BY 
        quantity DESC 
LIMIT   20